In [ ]:
! pip install -U accelerate
! pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 2.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import csv
from transformers import AutoTokenizer, PreTrainedTokenizerFast, GPT2LMHeadModel, Trainer, TrainingArguments, AutoModelForCausalLM, DataCollatorForLanguageModeling
import torch

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

# Tale.csv 파일 읽기
csv_file_path = "/content/drive/MyDrive/Tale/Tale.csv"
df = pd.read_csv(csv_file_path)

# 일부분 출력
print("Tale.csv 파일의 내용:")
print(df.head())  # 처음 5개 행 출력

# for idx, row in df.iterrows():
#   chapter = row['chap']
#   text = row['word']
#   print(text)

Tale.csv 파일의 내용:
   chap                             word
0     1        늑대가 양떼무리에서 떨어진 어린양을 만났어요.
1     1              늑대는 양을 잡아먹기로 작정했지요.
2     1  그래서 그렇듯 한 이야기를 하며 잡아먹을 생각을 했어요.
3     2                  늑대가 어린양에게 말했어요.
4     2              “어이, 네가 작년에 나 욕했지.”


In [ ]:
# traditional_tale.txt 파일 읽기
txt_file_path = "/content/drive/MyDrive/Tale/traditional_tale.txt"
with open(txt_file_path, "r", encoding="utf-8") as file:
    text = file.read()

# 일부분 출력
print("\ntraditional_tale.txt 파일의 내용:")
print(text[:500])  # 처음 500자 출력


traditional_tale.txt 파일의 내용:
옛날 옛적에 소망을 이루어 주는 힘이 여전히 통했던 때에, 딸들이 모두 예쁜 왕이 한 분 살고 계셨어요.
특히나 막내공주님이 아주 예뻤는데요, 어찌나 아름다운지, 해도 그녀의 얼굴에 햇볕을 비출 때마다 감탄을 하곤 하였더랬죠.
왕의 성은 울창한 숲에 둘러싸여 있었어요.
숲에 있는 나이 많은 라임 나무 아래에 우물이 하나 있었어요.
화창한 날에, 막내공주님께서 숲으로 놀러 나와 시원한 분수 옆에 앉았어요.
그녀는 따분할 땐 황금 공을 가져와 높이 던졌다가 다시 잡곤 했지요.
황금 공놀이는 그녀가 가장 좋아하는 놀이었어요.
그러다 우연히 공주의 작은 손으로 떠받치고 있던 황금 공이 바닥으로 떨어져 곧장 우물 속으로 굴러 들어갔지 뭐예요.
공주님이 다급히 눈으론 공을 쫓았지만 허사였어요.
우물은 꽤 깊었어요. 바닥이 보이지 않을 정도로요.
이 일로 공주님이 울기 시작했어요. 소리 내 울고 더 크게 소리 내 울었어요. 마음의 안정을 찾을 수가 없었어요.
그렇게 눈물을 글썽이고 있는데 누군가


In [ ]:
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>')
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

# special_tokens = ['<chap1>', '<chap2>', '<chap3>', '<chap4>', '</chap1>', '</chap2>', '</chap3>', '</chap4>']
# num_added_tokens = tokenizer.add_special_tokens({'additional_special_tokens': special_tokens})

# model.resize_token_embeddings(len(tokenizer))

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [ ]:
from torch.utils.data import Dataset, DataLoader

# def preprocess(row):
#     chapter = row['chap']
#     text = row['word']
#     source = f"챕터 {chapter}:"
#     target = f"<s>{text}</s>"
#     source_ids = tokenizer.encode(source, max_length=128, padding='max_length', truncation=True)
#     target_ids = tokenizer.encode(target, max_length=256, padding='max_length', truncation=True)
#     return source_ids, target_ids

# class StoriesDataset(Dataset):
#     def __init__(self, df, preprocess_fn):
#         self.df = df
#         self.preprocess_fn = preprocess_fn
#         self.encodings = [self.preprocess_fn(row) for _, row in self.df.iterrows()]

#     def __len__(self):
#         return len(self.encodings)

#     def __getitem__(self, idx):
#         return self.encodings[idx]

# dataset = StoriesDataset(df, preprocess_fn=preprocess)
# dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

# from transformers import DataCollatorForLanguageModeling
# data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)


# def preprocess(row):
#     text = row['word']
#     input_ids = tokenizer.encode(f"<s>{text}</s>", padding='longest', truncation=True)
#     return input_ids

# class StoriesDataset(Dataset):
#     def __init__(self, df, preprocess_fn):
#         self.df = df
#         self.preprocess_fn = preprocess_fn
#         self.encodings = [self.preprocess_fn(row) for _, row in self.df.iterrows()]

#     def __len__(self):
#         return len(self.encodings)

#     def __getitem__(self, idx):
#         return self.encodings[idx]

# dataset = StoriesDataset(df, preprocess_fn=preprocess)
# dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

# from transformers import DataCollatorForLanguageModeling

# data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# def preprocess_text(text):
#     chapters = []
#     current_chapter = []
#     for line in text.split('\n'):
#         if line.startswith('챕터'):
#             if current_chapter:
#                 chapters.append('\n'.join(current_chapter))
#                 current_chapter = []
#         else:
#             current_chapter.append(line)
#     if current_chapter:
#         chapters.append('\n'.join(current_chapter))

#     processed_text = '\n\n'.join(chapters)
#     processed_text = f"<s>{processed_text}</s>"
#     return processed_text

# def prepare_dataset(text, tokenizer, max_len=512):
#     processed_text = preprocess_text(text)
#     tokens = tokenizer.encode(processed_text, return_tensors="pt")

#     examples = tokens.squeeze().tolist()
#     examples = [example for example in examples if example != tokenizer.pad_token_id]

#     dataset = []
#     for i in range(len(examples) - max_len):
#         input_ids = examples[i:i+max_len]
#         labels = examples[i+1:i+max_len+1]
#         dataset.append((input_ids, labels))

#     return dataset

from transformers import PreTrainedTokenizerFast, DataCollatorForLanguageModeling

def preprocess_text(text):
    chapters = []
    current_chapter = []
    chapter_start = False
    for line in text.split('\n'):
        if line.startswith('챕터') and line.endswith('시작'):
            if current_chapter:
                chapters.append('\n'.join(current_chapter))
                current_chapter = []
            chapter_start = True
        elif line.startswith('챕터') and line.endswith('끝'):
            current_chapter.append(line)
            chapters.append('\n'.join(current_chapter))
            current_chapter = []
            chapter_start = False
        elif chapter_start:
            current_chapter.append(line)

    processed_text = '\n\n'.join(chapters)
    processed_text = f"<s>{processed_text}</s>"
    return processed_text

def prepare_dataset(text, tokenizer, max_len=512):
    processed_text = preprocess_text(text)
    tokens = tokenizer.encode(processed_text, return_tensors="pt")

    examples = tokens.squeeze().tolist()
    examples = [example for example in examples if example != tokenizer.pad_token_id]

    dataset = []
    for i in range(len(examples) - max_len):
        input_ids = examples[i:i+max_len]
        labels = examples[i+1:i+max_len+1]
        dataset.append((input_ids, labels))

    return dataset

def get_data_collator(tokenizer):
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False, pad_to_multiple_of=8, truncation=True)
    return data_collator

def collate_batch(data_collator, dataset, max_len=512):
    input_ids = torch.tensor([ex[0] for ex in dataset])
    labels = torch.tensor([ex[1] for ex in dataset])
    batch = data_collator(input_ids, labels=labels, max_length=max_len)
    return batch

tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2")
dataset = prepare_dataset(text_content, tokenizer)
data_collator = get_data_collator(tokenizer)

batch = collate_batch(data_collator, dataset[:10], max_len=512)
print(batch)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


TypeError: DataCollatorForLanguageModeling.__init__() got an unexpected keyword argument 'truncation'

In [ ]:
import re

def preprocess_text(text):
    # 문장 분리
    sentences = re.split(r'(?<=[.!?]) +', text)

    processed = []
    for sentence in sentences:
        # 시퀀스 길이 제한
        if len(tokenizer.encode(sentence)) > 512:
            chunks = [sentence[i:i+250] for i in range(0, len(sentence), 250)]
            processed.extend(chunks)
        else:
            processed.append(sentence)

    processed_text = ' '.join(processed)
    return processed_text

processed_text = preprocess_text(text)

input_ids = tokenizer.encode(processed_text, return_tensors='pt')[0]
dataset = torch.utils.data.TensorDataset(input_ids)


data_loader = torch.utils.data.DataLoader(dataset, batch_size=4, shuffle=True)


from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

# Define data collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Define training arguments
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/kogpt2-story-generator',    # Output directory
    num_train_epochs=3,              # Total number of training epochs
    per_device_train_batch_size=8,   # Batch size per device during training
    save_steps=500,                  # Save checkpoint every N updates steps
    save_total_limit=2,              # Limit the total amount of checkpoints
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
)

# Start training
trainer.train()

# Save the trained model
trainer.save_model()


Step,Training Loss
500,0.000000
1000,0.000000
1500,0.000000
2000,0.000000
2500,0.000000
3000,0.000000
3500,0.000000
4000,0.000000
4500,0.000000
5000,0.000000


KeyboardInterrupt: 

In [ ]:
from transformers import Trainer, TrainingArguments

# 훈련을 위한 설정 정의
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/kogpt2-story-generator',          # 출력 디렉토리
    num_train_epochs=3,              # 총 훈련 에폭 수
    per_device_train_batch_size=8,   # 배치 크기
    save_steps=500,                  # 일정 스텝마다 모델 저장
    save_total_limit=2,              # 최대 저장 모델 수
)

trainer = Trainer(
    model = model,
    args = training_args,
    data_collator = data_collator,
    train_dataset = dataset
)
trainer.train()
trainer.save_model()

ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`labels` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

In [ ]:
from transformers import PreTrainedTokenizerFast, DataCollatorForLanguageModeling
import torch

def prepare_dataset(text, tokenizer, max_len=512):
    chapters = []
    for line in text.split('\n'):
        if line.startswith('챕터') and line.endswith('시작'):
            chapters.append([])
        elif line.startswith('챕터') and line.endswith('끝'):
            continue
        else:
            chapters[-1].append(line)

    processed_text = '\n\n'.join(['\n'.join(chapter) for chapter in chapters])
    processed_text = f"<s>{processed_text}</s>"

    tokens = tokenizer.encode(processed_text, return_tensors="pt")
    examples = tokens.squeeze().tolist()
    examples = [example for example in examples if example != tokenizer.pad_token_id]

    dataset = []
    for i in range(len(examples) - max_len):
        input_ids = examples[i:i+max_len]
        labels = examples[i+1:i+max_len+1]
        dataset.append((input_ids, labels))

    return dataset

def get_data_collator(tokenizer):
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False, pad_to_multiple_of=8, truncation=True)
    return data_collator

def collate_batch(data_collator, dataset, max_len=512):
    input_ids = torch.tensor([ex[0] for ex in dataset])
    labels = torch.tensor([ex[1] for ex in dataset])
    batch = data_collator(input_ids, labels=labels, max_length=max_len)
    return batch

tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2")
dataset = prepare_dataset(text_content, tokenizer)
data_collator = get_data_collator(tokenizer)

batch = collate_batch(data_collator, dataset[:10], max_len=512)
print(batch)

from transformers import PreTrainedTokenizerFast, DataCollatorForLanguageModeling, GPT2LMHeadModel, Trainer, TrainingArguments
import torch

# 데이터 준비
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2")
dataset = prepare_dataset(text_content, tokenizer)
data_collator = get_data_collator(tokenizer)

# 모델 및 학습 인자 설정
model = GPT2LMHeadModel.from_pretrained("skt/kogpt2-base-v2")
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    save_total_limit=3,
    prediction_loss_only=True,
)

# Trainer 초기화
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=data_collator,
)

# 학습 진행
trainer.train()

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


AttributeError: 'Tensor' object has no attribute 'pad'

In [ ]:
from transformers import PreTrainedTokenizerFast, DataCollatorForLanguageModeling
import torch
import re

# 텍스트 데이터를 챕터 단위로 분리
chapter_pattern = r'<chap>(.*?)</chap>'
chapters = re.findall(chapter_pattern, text, re.DOTALL)

# 데이터셋 생성
dataset = [tokenizer(chapter, truncation=True, max_length=512, return_tensors="pt") for chapter in chapters]

# 데이터 컬렉터 생성
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)


from transformers import Trainer, TrainingArguments

# 훈련을 위한 설정 정의
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/kogpt2-story-generator',          # 출력 디렉토리
    num_train_epochs=3,              # 총 훈련 에폭 수
    per_device_train_batch_size=8,   # 배치 크기
    save_steps=500,                  # 일정 스텝마다 모델 저장
    save_total_limit=2,              # 최대 저장 모델 수
)

trainer = Trainer(
    model = model,
    args = training_args,
    data_collator = data_collator,
    train_dataset = dataset
)
trainer.train()
trainer.save_model()

텍스트 데이터에서 챕터를 찾을 수 없습니다.


ValueError: num_samples should be a positive integer value, but got num_samples=0

In [ ]:
from transformers import pipeline

# Initialize text generation pipeline
text_generation_pipeline = pipeline(
    "text-generation",
    model="/content/drive/MyDrive/kogpt2-story-generator",  # Path to the trained model
    tokenizer="skt/kogpt2-base-v2",
    device=0  # Set device=0 to use GPU, -1 to use CPU
)

# Generate a chapter
generated_chapter = text_generation_pipeline(
    "왕자는 공주를 구하기 위해 모험을 떠났어요. ",
    max_length=500,
    min_length=300,
    num_return_sequences=1,
    truncation=True
)

print(generated_chapter[0]['generated_text'])


OSError: Incorrect path_or_model_id: '/content/drive/MyDrive/kogpt2-story-generator/checkpoing-10000'. Please provide either the path to a local folder or the repo_id of a model on the Hub.